In [1]:
%matplotlib

import sys
sys.path.append('..')

from brian2 import defaultclock, ms, pA, nA, Hz, seed, mV, second
import numpy as np
from results import Results
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

import seaborn as sns
import pandas as pd

Using matplotlib backend: Qt5Agg


INFO       Cache size for target "cython": 15612 MB.
You can call "clear_cache('cython')" to delete all files from the cache or manually delete files in the "C:\Users\mikejseay\.cython\brian_extensions" directory. [brian2]


In [11]:
# targetFile = 'buonoEphys_2000_0p25_upPoisson_guessBuono2Weights2e3p025LogNormal_test_2021-08-13-14-55_results'  # very nice but not the right params exactly
# targetFile = 'buonoEphys2_2000_0p25_upPoisson_guessBuono2Weights2e3p025LogNormal2_test_2021-08-16-14-26_results'  # very nice
# targetFile = 'buonoEphys2_2000_0p25_upPoisson_guessBuono2Weights2e3p025LogNormal2_test_2021-08-16-14-35_results'  # very nice
# targetFile = 'buonoEphysBen1_2000_0p25_upPoisson_guessBuono2Weights2e3p025LogNormal_test_2021-08-16-16-19_results'  # very nice (ben idea worked)
# targetFile = 'buonoEphysBen1_2000_0p25_upPoisson_guessBuono2Weights2e3p025LogNormal_test_2021-08-16-16-46_results'  # another example
# targetFile = 'buonoEphysBen1_2000_0p25_upPoisson_guessBuono2Weights2e3p025LogNormal_test_2021-08-17-11-09_results'  # good

# targetFile = 'buonoEphysBen1_2000_0p25_upPoisson_guessBuono2Weights2e3p025LogNormal_test_2021-08-17-14-13_results'  # no Exc2
# targetFile = 'buonoEphysBen1_2000_0p25_upPoisson_guessBuono2Weights2e3p025LogNormal_test_2021-08-17-14-16_results'  # yes Exc2

# targetFile = 'buonoEphysBen1_2000_0p25_upPoisson_guessBuono4Weights2e3p025LogNormal3__2021-08-23-14-49_results'  # yes Exc2
# targetFile = 'buonoEphysBen1_2000_0p25_upPoisson_guessBuono4Weights2e3p025LogNormal3__2021-08-23-14-12_results'  # no Exc2

# targetFile = 'buonoEphysBen1_2000_0p5_upPoisson_guessBuono4Weights2e3p05LogNormal3__2021-08-23-15-57_results'  # yes Exc2
# targetFile = 'buonoEphysBen1_2000_0p5_upPoisson_guessBuono4Weights2e3p05LogNormal3__2021-08-23-15-52_results'  # no Exc2

# targetFile = 'buonoEphysBen1_2000_0p5_upPoisson_guessBuono4Weights2e3p05LogNormal3__2021-08-23-16-05_results'  # yes Exc2, really good

# targetFile = 'buonoEphysBen1_2000_0p5_upPoisson_guessBuono6Weights2e3p05Beta10__2021-08-24-11-16_results'  # no Exc2
# targetFile = 'buonoEphysBen1_2000_0p5_upPoisson_guessBuono6Weights2e3p05Beta10__2021-08-24-11-20_results'  # yes Exc2

# targetFile = 'buonoEphysBen1_2000_0p25_upPoisson_resumePrior__2021-09-04-17-21_results'  # best so far
# targetFile = 'buonoEphysBen1_2000_0p25_upPoisson_resumePrior__2021-09-05-21-49_results'  # control equivalent

# targetFile = 'buonoEphysBen1_2000_0p25_upPoisson_resumePrior__2021-09-08-14-28_results'  # good control, 30 s with 2 ups
# targetFile = 'buonoEphysBen1_2000_0p25_upPoisson_resumePrior__2021-09-08-14-43_results'  # good experimental, 30 s with 2 ups

# targetFile = 'buonoEphysBen1_2000_0p25_upPoisson_resumePrior__2021-09-09-10-47_results'  # modify conn 1: 10% removed
targetFile = 'buonoEphysBen1_2000_0p25_upPoisson_resumePrior__2021-09-10-10-15_results'  # modify conn 2: 10% removed, some added back

# targetFile = 'buonoEphysBen1_2000_0p25_upPoisson_resumePrior__2021-10-04-13-54_results'  # 20% removed, added back
# targetFile = 'buonoEphysBen1_2000_0p25_upPoisson_resumePrior_EImod_2021-10-04-14-03_results'  # 20% removed, added back

targetFolder = 'C:/Users/mikejseay/Documents/BrianResults/'

In [12]:
R = Results()
R.init_from_file(targetFile, targetFolder)

print(R.p['simName'])

classicJercogEphysBuonoBen1


In [13]:
plt.close('all')

In [14]:
R.p['indsRecordStateExc']

[0, 1599, 100, 940, 1052, 1434]

In [15]:
R.p['indsRecordStateInh']

[0, 27]

In [16]:
R.calculate_PSTH()
R.calculate_voltage_histogram(removeMode=True, removeReset=True, useAllRecordedUnits=False, useExcUnits=1)
R.calculate_upstates()
if len(R.ups) > 0:
    R.reshape_upstates()
    R.calculate_FR_in_upstates()
    infoStr = 'average FR in upstate for Exc: {:.2f}, Inh: {:.2f}, average upDur: {:.2f}, upFreq: {:.2f}'.format(R.upstateFRExcHist.mean(), R.upstateFRInhHist.mean(), R.upDurs.mean(), R.ups.size / R.p['duration'] / Hz)
    print(infoStr)


R.calculate_voltage_histogram(useExcUnits=(2, 4,), useInhUnits=(0,))
R.reshape_upstates()

average FR in upstate for Exc: 3.87, Inh: 9.14, average upDur: 0.77, upFreq: 0.13


In [9]:
# special plot: two voltages for a 20 sec period with 3 up states (hopefully)
# e vs i

timePeriod = (0, 8)

f, ax = plt.subplots(2, 1, num=2, figsize=(16, 5), sharex=True, sharey=True)
R.plot_voltage_detail(ax[0], unitType='Exc', useStateInd=3)
R.plot_voltage_detail(ax[1], unitType='Inh', useStateInd=1)
R.plot_updur_lines(ax[1])
ax[1].set(xlim=timePeriod, xlabel='Time (s)')

f.savefig('benEphys1Voltage.pdf', transparent=True)

In [33]:
# special plot: two voltages for a 20 sec period with 3 up states (hopefully)
# e- vs e+

timePeriod = (49, 58)

f, ax = plt.subplots(2, 1, num=2, figsize=(9, 5), sharex=True, sharey=True)
R.plot_voltage_detail(ax[0], unitType='Exc', useStateInd=3)
R.plot_voltage_detail(ax[1], unitType='Exc', useStateInd=2, overrideColor='royalblue')
R.plot_updur_lines(ax[1])
ax[1].set(xlim=timePeriod, xlabel='Time (s)')

f.savefig('benEphys3Voltage.pdf', transparent=True)

In [10]:
# special plot: spike raster for a 20 sec period with 3 up states (hopefully)

timePeriod = (0, 8)

fig1, ax1 = plt.subplots(2, 1, num=3,
                         gridspec_kw={'height_ratios': [2, 1]},
                         figsize=(16, 5), sharex=True)
R.plot_spike_raster(ax1[0])
R.plot_firing_rate(ax1[1])
ax1[1].set_ylim(0, 40)
ax1[1].set(xlim=timePeriod, xlabel='Time (s)')

fig1.savefig('benEphys1Raster.pdf', transparent=True)

In [17]:
fig1, ax1 = plt.subplots(6, 1, num=1, figsize=(16, 9),
                         gridspec_kw={'height_ratios': [3, 1, 1, 1, 1, 1]},
                         sharex=True)
R.plot_spike_raster(ax1[0])  # uses RNG but with a separate random seed
R.plot_firing_rate(ax1[1])
ax1[1].set_ylim(0, 30)
R.plot_voltage_detail(ax1[2], unitType='Exc', useStateInd=0)
R.plot_updur_lines(ax1[2])
R.plot_voltage_detail(ax1[3], unitType='Inh', useStateInd=0)
R.plot_updur_lines(ax1[3])
R.plot_voltage_detail(ax1[4], unitType='Exc', useStateInd=1)
R.plot_updur_lines(ax1[4])
if 'useSecondPopExc' in R.p and R.p['useSecondPopExc']:
    R.plot_voltage_detail(ax1[5], unitType='Exc', useStateInd=2)
    R.plot_updur_lines(ax1[5])
ax1[5].set(xlabel='Time (s)')
R.plot_voltage_histogram_sideways(ax1[2], 'Exc')
R.plot_voltage_histogram_sideways(ax1[3], 'Inh')
fig1.suptitle(infoStr)

Text(0.5, 0.98, 'average FR in upstate for Exc: 3.87, Inh: 9.14, average upDur: 0.77, upFreq: 0.13')

In [46]:
fig1, ax1 = plt.subplots(5, 1, num=1, figsize=(16, 9),
                         gridspec_kw={'height_ratios': [3, 1, 1, 1, 1]},
                         sharex=True)
R.plot_spike_raster(ax1[0])  # uses RNG but with a separate random seed
R.plot_firing_rate(ax1[1])
ax1[1].set_ylim(0, 30)
R.plot_voltage_detail(ax1[2], unitType='Exc', useStateInd=5)
R.plot_voltage_detail(ax1[3], unitType='Inh', useStateInd=0)
R.plot_voltage_detail(ax1[4], unitType='Exc', useStateInd=2, overrideColor='royalblue')
R.plot_updur_lines(ax1[2])
R.plot_updur_lines(ax1[3])
R.plot_updur_lines(ax1[4])
ax1[4].set(xlabel='Time (s)')
R.plot_voltage_histogram_sideways(ax1[2], 'Exc')
R.plot_voltage_histogram_sideways(ax1[3], 'Inh')
R.plot_voltage_histogram_sideways(ax1[4], 'Exc')
fig1.suptitle(R.p['simName'])

Text(0.5, 0.98, 'classicJercogEphysBuonoBen1')

In [43]:
R.calculate_voltage_histogram(removeMode=True, useAllRecordedUnits=False, useExcUnits=2)

In [15]:
f, ax = plt.subplots(figsize=(6, 4))
# R.plot_voltage_histogram(ax)
# ax.plot(R.voltageHistCenters, (R.voltageHistExc + R.voltageHistInh)/2)
ax.plot(R.voltageHistCenters, R.voltageHistExc)
ax.set(xlabel='Voltage (mv)', ylabel='# of occurences')

[Text(0, 0.5, '# of occurences'), Text(0.5, 0, 'Voltage (mv)')]

In [21]:
R.calculate_upFR_units()

In [22]:
frInp = R.upstateFRExcUnits[:, :R.p['nUnitsToSpike']].mean(0)  # input pop
frEx2 = R.upstateFRExcUnits[:, R.p['nUnitsToSpike']:(R.p['nUnitsToSpike'] + R.p['nUnitsSecondPopExc'])].mean(0)  # secondary pop
frEx1 = R.upstateFRExcUnits[:, (R.p['nUnitsToSpike'] + R.p['nUnitsSecondPopExc']):].mean(0)  # normal pop
frInh = R.upstateFRInhUnits.mean(0)

frInpHat = frInp.mean()
frEx2Hat = frEx2.mean()
frEx1Hat = frEx1.mean()
frInhHat = frInh.mean()

frInpSTD = frInp.std()
frEx2STD = frEx1.std()
frEx1STD = frEx1.std()
frInhSTD = frInh.std()

print(frInpHat, frEx2Hat, frEx1Hat, frInhHat, )
print(frInpSTD, frEx2STD, frEx1STD, frInhSTD, )

frInpDF = pd.DataFrame(frInp, columns=('FR',))
frEx2DF = pd.DataFrame(frEx2, columns=('FR',))
frEx1DF = pd.DataFrame(frEx1, columns=('FR',))
frInhDF = pd.DataFrame(frInh, columns=('FR',))

frInpDF['unitType'] = 'Inp'
frEx2DF['unitType'] = 'Ex2'
frEx1DF['unitType'] = 'Ex1'
frInhDF['unitType'] = 'Inh'

frDF = pd.concat((frInpDF, frEx2DF, frEx1DF, frInhDF, ))

# frDF = pd.concat((frEx2DF, frEx1DF, frInhDF, ))

4.201520772828711 0.3603609544758373 5.366307916111041 12.931081976292495
0.6167987428108929 0.7335771194929483 0.7335771194929483 2.157713573405929


In [23]:
frEx1C = R.upstateFRExcUnits[:, (R.p['nUnitsToSpike'] + R.p['nUnitsSecondPopExc']):].mean(0)  # normal pop
frEx1CHat = frEx1C.mean()
frEx1CSTD = frEx1C.std()
frEx1CDF = pd.DataFrame(frEx1C, columns=('FR',))
frEx1CDF['unitType'] = 'Ex1C'

In [24]:
frDF = pd.concat((frInpDF, frEx2DF, frEx1DF, frInhDF, frEx1CDF))

# frDF = pd.concat((frEx2DF, frEx1DF, frInhDF, ))

In [51]:
frDF

,FR,unitType
0,4.000377,Inp
1,4.089369,Inp
2,4.588998,Inp
3,3.576164,Inp
4,3.288823,Inp
...,...,...
1395,3.988800,Ex1C
1396,4.458446,Ex1C
1397,4.445625,Ex1C
1398,4.431041,Ex1C


In [27]:
# Create an array with the colors you want to use
# colors = ["#006400", "#90EE90"]
# colors = ["#006400", "#00FFFF"]
colors = ["#00FFFF", "#4E69B2"]
# colors = ["#006400", "#90EE90", "#00FFFF"]
#00FFFF

# Set your custom color palette
customPalette = sns.set_palette(sns.color_palette(colors))

In [28]:
f, ax = plt.subplots(figsize=(3.25, 4))
# sns.violinplot(x='unitType', y='FR', data=frDF, width=.6, palette=customPalette, order=('Ex1C', 'Ex1', 'Ex2'))
sns.violinplot(x='unitType', y='FR', data=frDF, width=.6, palette=customPalette, order=('Ex1', 'Ex2'))
# sns.stripplot(x="unitType", y="FR", data=frDF, size=4, color=".3", linewidth=0)
f.savefig('benEphys3Violin.pdf', transparent=True)

In [60]:
sns.swarmplot(x='unitType', y='FR', size=1.5, color='k', data=frDF, order=('Ex1', 'Ex2'))

In [64]:
f, ax = plt.subplots()
sns.boxplot(x='unitType', y='FR', data=frDF, width=.6, order=('Ex1', 'Ex2'), palette=customPalette)
# sns.stripplot(x="unitType", y="FR", data=frDF, size=4, color=".3", linewidth=0)

In [63]:
sns.stripplot(x='unitType', y='FR', size=1.5, color='k', data=frDF, order=('Ex1', 'Ex2'))

In [22]:
# get the QTCore PyRect object
geom = mngr.window.geometry()

x,y,dx,dy = geom.getRect()

print(x,y,dx,dy)

1997 83 640 532


In [ ]:
mngr = plt.get_current_fig_manager()
# to put it into the upper left corner for example:
# mngr.window.setGeometry(50,100,640, 545)
targetFile = ''